# MAGIC + LST combined analysys: stereo parameter reconstruction

In [1]:
import yaml
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import magicctapipe
from IPython.display import Image
print(f'magicctapipe version: {magicctapipe.__version__}')

magicctapipe version: 0.3.0


In [2]:
# --- display all columns --- 
pd.set_option('display.max_columns', None)

# --- customize plt figure ---
plt.rcParams['figure.figsize'] = (12, 9)
plt.rcParams['font.size'] = 15
plt.rcParams['grid.linestyle'] = ':'

# --- get the default color cycle ---
color_cycle = plt.rcParams['axes.prop_cycle'].by_key()['color']

The script for the stereo parameter reconstruction is <b>lst1_magic_stereo_reco.py<b>
    
This script processes DL1 events and reconstructs the geometrical stereo parameters with more than one telescope information. The quality cuts specified in the configuration file are applied to the events before the reconstruction. When the input is real data containing LST-1 and MAGIC events, it checks the angular distances of their pointing directions and excludes theevents taken with larger distances than the limit specified in the configuration file. This is in principle to avoid the reconstruction ofthe events taken in too-mispointing situations. For example, DL1 data may contain the coincident events taken with different wobble offsets between the systems. If the `--magic-only` argument is given, it reconstructs the stereoparameters using only MAGIC events.

<img src="Stereo_tech.png" alt="Alternative text" />
    
<b>USAGE<b>:

python lst1_magic_stereo_reco.py
    
--input-file dl1_LST-1_MAGIC.Run03265.0040.h5
    
(--output-dir dl1_stereo)
    
(--config-file config.yaml)
    
(--magic-only)

# 1) Input data

if you work at the IT container:

In [3]:
#school_data_dir='/fefs/aswg/workspace/2023_joint_analysis_school'

if you work at pic

In [4]:
#school_data_dir='pnfs/pic.es/data/magic/Users/SWschool2023/data/2023_joint_analysis_school'

Locally:

In [5]:
school_data_dir='/home/gpirola/Desktop/SoftwareSchool2023/2023_joint_analysis_school'

In [6]:
date='20210213'

In [7]:
dl1_coincidence_dir=f'{school_data_dir}/stereo_reconstruction/input/DL1_coinc'

In [8]:
input_file = f'{dl1_coincidence_dir}/{date}/dl1_LST-1_MAGIC.Run03636.0039.h5'

# 2) Config file

In [9]:
config_file = './config_MAGIC_LST.yaml'

with open(config_file, 'rb') as f:
    config = yaml.safe_load(f)

In [10]:
config['stereo_reco']

{'quality_cuts': '(intensity > 50) & (width > 0)', 'theta_uplim': '6 arcmin'}

    'theta_uplim':

    it's an upper limit on the angular pointing difference between the telescopes (6 arcmin=0.1 deg)

# 3) Running the script

In [11]:
output_dir = './stereo/output'
!mkdir -p $output_dir

Execute script from terminal:

In [ ]:
!lst1_magic_stereo_reco\
--input-file $input_file\
--output-dir $output_dir\
--config-file $config_file


Input file: /home/gpirola/Desktop/SoftwareSchool2023/2023_joint_analysis_school/stereo_reconstruction/input/DL1_coinc/20210213/dl1_LST-1_MAGIC.Run03636.0039.h5

Is simulation: False

Telescope positions:
    1: <Quantity  -8.09 77.13  0.78  m>
    2: <Quantity   39.3  -62.55  -0.97  m>
    3: <Quantity  -31.21 -14.57   0.2   m> 

MAGIC-only analysis: False

Quality cuts: (intensity > 50) & (width > 0)

In total 2254 stereo events are found:
    M1_M2 (type 0): 374 events (16.6%)
    LST1_M1 (type 1): 137 events (6.1%)
    LST1_M2 (type 2): 327 events (14.5%)
    LST1_M1_M2 (type 3): 1416 events (62.8%) 

Checking the angular distances of the LST-1 and MAGIC pointing directions...
--> All the events were taken with smaller angular distances than the limit 6.0 arcmin.

Reconstructing the stereo parameters...
0 events
100 events
200 events
300 events
400 events
500 events
600 events
700 events
800 events
900 events
1000 events
1100 events
1200 events
1300 events
1400 events
1500 events
1

Import function on a notebook:

In [ ]:
from magicctapipe.scripts.lst1_magic import lst1_magic_stereo_reco

In [ ]:
#lst1_magic_stereo_reco.stereo_reconstruction(input_file, output_dir, config)

# 4) Check the output data

In [ ]:
output_file=f'{output_dir}/dl1_stereo_LST-1_MAGIC.Run03636.0039.h5'

In [ ]:
data = pd.read_hdf(output_file, key='events/parameters')
data.set_index(['obs_id', 'event_id', 'tel_id'], inplace=True)
data.sort_index(inplace=True)

In [ ]:
data

# 5) Merge the subrub files

The script for merging hdf file is <b>merge_hdf_files.py<b>


This script merges the HDF files produced by the LST-1 + MAGIC combined analysis pipeline. It parses information from the file names, so they should follow the convention, i.e., *Run*.*.h5 or *run*.h5. If no output directory is specified with the `--output-dir` argument, it saves merged files in the `merged` directory which will be created
under the input directory. If the `--run-wise` argument is given, it merges input files run-wise.
It is applicable only to real data since MC data are already produced run-wise. The `--subrun-wise` argument can be also used to merge MAGIC DL1 real data subrun-wise (for example, dl1_M1.Run05093711.001.h5 + dl1_M2.Run05093711.001.h5 -> dl1_MAGIC.Run05093711.001.h5).

<b>Usage<b>:
python merge_hdf_files.py

--input-dir dl1

(--output-dir dl1_merged)

(--run-wise)

(--subrun-wise)

In [ ]:
stereo_subrun_dir=f'{school_data_dir}/stereo_reconstruction/input/DL1_stereo/{date}'
stereo_merged_dir=f'{output_dir}/merged'
!mkdir -p $stereo_merged_dir

# 5.1) Running the script

Execute script from terminal:

In [ ]:
!merge_hdf_files\
--input-dir $stereo_subrun_dir\
--output-dir $stereo_merged_dir\
--run-wise

Import function on a notebook:

In [ ]:
from magicctapipe.scripts.lst1_magic import merge_hdf_files

In [ ]:
#merge_hdf_files(input_dir, output_dir)
#without specifying the run_wise flag to be true, it will merge all the subruns in the direcotry togheter

Check the content of the merged file:

In [ ]:
stereo_merged_file=f'{stereo_merged_dir}/dl1_stereo_LST-1_MAGIC.Run03636.h5'

In [ ]:
data_merged = pd.read_hdf(stereo_merged_file, key='events/parameters')
data_merged.set_index(['obs_id', 'event_id', 'tel_id'], inplace=True)
data_merged.sort_index(inplace=True)

In [ ]:
data_merged

# 5.2) Check the stereo parameter distributions

In [ ]:
plt.figure(figsize=(16,7))

m2km = 1e-3
plt.subplot(121)
df = data_merged.query(f'multiplicity == 3').groupby(['obs_id', 'event_id']).mean()

plt.hist(
    df['h_max'].to_numpy() * m2km, bins=np.linspace(0, 20, 41), label='LST1+M1+M2'
)
plt.title(f'H$_{{max}}$ distribution')
plt.xlabel(f'H$_{{max}}$ [km]')
plt.ylabel('Number of events')
plt.yscale('log')
plt.legend()
plt.grid()

plt.subplot(122)
plt.hist(
    df['impact'].to_numpy(), bins=np.linspace(0, 2000, 100), label='LST1+M1+M2'
)
plt.title(f'Average Impact parameter distribution')
plt.xlabel('Impact parameter [m]')
plt.ylabel('Number of events')
plt.xlim(0,1500)  
plt.yscale('log')
plt.legend()
plt.grid()

# 6) Check the core position distribution

Extraction of the geometrical position of the telescopes in the simtel array system

In [ ]:
geo_info = pd.read_hdf(stereo_merged_file, key='configuration/instrument/subarray/layout')
geo_info

In [ ]:
LST1_coords=[geo_info[geo_info.tel_id==1].pos_x,geo_info[geo_info.tel_id==1].pos_y]
MAGIC1_coords=[geo_info[geo_info.tel_id==2].pos_x,geo_info[geo_info.tel_id==2].pos_y]
MAGIC2_coords=[geo_info[geo_info.tel_id==3].pos_x,geo_info[geo_info.tel_id==3].pos_y]

In [ ]:
combo_names=['M1+M2','LST1+M1','LST1+M2','LST1+M1+M2']
bins=[100,100]
plt.figure(figsize=(16,14))
plot_range=[-150,150]
for i,combo_name in enumerate(combo_names):
    a=221+i
    plt.subplot(a)
    plt.title(f'{combo_name}')

    ax = plt.gca()
    df = data_merged.query(f'combo_type=={i}').groupby(['obs_id', 'event_id']).mean()

    plt.hist2d(
        df['core_x'].to_numpy(),
        df['core_y'].to_numpy(),
        range=[[plot_range[0],plot_range[1]], [plot_range[0],plot_range[1]]],
        bins=bins,norm='log')



    circle_1=plt.Circle([a for a in MAGIC1_coords],17,color='r', fill=False,label='M1')
    circle_2=plt.Circle([a for a in MAGIC2_coords],17,color='r', fill=False,label='M2')
    circle_L=plt.Circle([a for a in LST1_coords],23,color='black', fill=False,label='LST-1')
    ax.add_patch(circle_1)
    ax.add_patch(circle_2)
    ax.add_patch(circle_L)
    plt.text(MAGIC1_coords[0],MAGIC1_coords[1],s='M1',color='r')
    plt.text(MAGIC2_coords[0],MAGIC2_coords[1],s='M2',color='r')
    plt.text(LST1_coords[0],LST1_coords[1],s='LST-1',color='black')

    plt.xlabel('Core X (S->N) [m]',fontsize=14)
    plt.ylabel('Core Y (E->W) [m]',fontsize=14)
    plt.xlim(plot_range[0],plot_range[1])
    plt.ylim(plot_range[0],plot_range[1])
    plt.colorbar()
    plt.grid()